# Healthcare Patient Monitoring Hub - Interactive Dashboard

## Executive Summary

### Chosen Scenario
This project implements a **Healthcare Patient Monitoring Hub** for a hospital wing that monitors patient vitals and nurse/doctor shift allocations. The system tracks real-time patient health metrics, medication administration, and staff workload distribution to identify high-risk patients and optimize healthcare delivery.

### Database Schema (ER Diagram Description)

The database consists of **4 related tables** forming a comprehensive relational schema:

#### 1. **Doctors Table**
- **Primary Key**: `doctor_id` (INTEGER, AUTO INCREMENT)
- **Attributes**: `name`, `specialty`, `shift_start`, `shift_end`, `years_experience`
- **Purpose**: Stores medical staff information and shift schedules

#### 2. **Patients Table**
- **Primary Key**: `patient_id` (INTEGER, AUTO INCREMENT)
- **Attributes**: `name`, `age`, `gender`, `admission_date`, `diagnosis`
- **Foreign Key**: `doctor_id` → `Doctors(doctor_id)`
- **Purpose**: Contains patient demographics and links to assigned doctor

#### 3. **VitalsLog Table**
- **Primary Key**: `vitals_id` (INTEGER, AUTO INCREMENT)
- **Attributes**: `timestamp`, `heart_rate`, `blood_pressure`, `temperature`, `oxygen_saturation`
- **Foreign Key**: `patient_id` → `Patients(patient_id)`
- **Purpose**: Time-series data tracking patient vital signs over time

#### 4. **Medications Table**
- **Primary Key**: `med_id` (INTEGER, AUTO INCREMENT)
- **Attributes**: `medication_name`, `dosage`, `frequency`, `administered_date`
- **Foreign Keys**: 
  - `patient_id` → `Patients(patient_id)`
  - `prescribed_by` → `Doctors(doctor_id)`
- **Purpose**: Tracks medication prescriptions and administration

### Entity Relationships
```
Doctors (1) ────── (Many) Patients
    │                        │
    │                        ├─── (Many) VitalsLog
    │                        │
    └────── (Many) Medications ──┘
```

### Key Features
- **Data Volume**: 1,000 patients, 15,000 vitals records, 5,000 medications
- **SQL Complexity**: Window functions, 3-way JOINs, aggregations, subqueries
- **Visualizations**: Interactive dashboard with real-time filtering
- **Healthcare Focus**: High-risk patient detection and shift workload analysis

---

In [ ]:
# ============================================================================
# CELL 1: INSTALL REQUIRED PYTHON PACKAGES
# ============================================================================
# This cell installs all necessary libraries for the Healthcare Patient Monitoring Hub.
# 
# Package purposes:
# - pandas: Data manipulation and analysis with DataFrames
# - numpy: Numerical computing and random data generation
# - matplotlib: Basic plotting and chart creation
# - seaborn: Statistical data visualization
# - plotly: Interactive plots and dashboards
# - ipywidgets: Interactive widgets for Jupyter notebooks
# 
# Note: SQLite3 is built into Python, no installation needed
# ============================================================================

!pip install -q pandas numpy matplotlib seaborn plotly ipywidgets
print("✓ All packages installed successfully!")

In [ ]:
# ============================================================================
# CELL 2: IMPORT ALL NECESSARY LIBRARIES
# ============================================================================
# Importing all required libraries for database operations, data manipulation,
# and visualization in the Healthcare Patient Monitoring Hub.
# ============================================================================

# Database and core Python libraries
import sqlite3  # SQLite database operations - lightweight relational database
from datetime import datetime, timedelta  # Date and time manipulation
import random  # Random number generation for synthetic data

# Data manipulation and analysis
import pandas as pd  # DataFrames and data manipulation
import numpy as np  # Numerical operations and array handling

# Visualization libraries
import matplotlib.pyplot as plt  # Basic plotting functionality
import seaborn as sns  # Statistical data visualization
import plotly.express as px  # High-level interactive plotting
import plotly.graph_objects as go  # Advanced customizable plotly figures
from plotly.subplots import make_subplots  # Creating subplots in plotly

# Interactive widgets for dashboard
import ipywidgets as widgets  # Interactive controls (dropdowns, sliders, etc.)
from IPython.display import display, clear_output, HTML  # Display management

# Configure visualization settings
sns.set_style('whitegrid')  # Set seaborn style for better-looking plots
plt.rcParams['figure.figsize'] = (12, 6)  # Default figure size

print("✓ All libraries imported successfully!")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ Numpy version: {np.__version__}")

In [ ]:
# ============================================================================
# CELL 3: DATABASE SCHEMA CREATION
# ============================================================================
# This cell defines and creates the relational database schema for the
# Healthcare Patient Monitoring Hub. The schema consists of 4 related tables:
# 1. Doctors - Medical staff information and shift schedules
# 2. Patients - Patient demographics and admission details
# 3. VitalsLog - Time-series vital sign measurements
# 4. Medications - Prescribed medications and administration records
# ============================================================================

def create_database_schema(db_path='healthcare.db'):
    """
    Creates the complete SQLite database schema for healthcare monitoring.
    
    This function establishes a relational database with proper foreign key
    relationships between tables. It includes error handling and ensures
    idempotency (can be run multiple times safely).
    
    Args:
        db_path (str): File path for the SQLite database. Defaults to 'healthcare.db'
    
    Returns:
        None
    
    Tables Created:
        - Doctors: Stores doctor information and shift timings
        - Patients: Contains patient demographics and assigned doctor
        - VitalsLog: Time-series data of patient vital signs
        - Medications: Medication prescriptions and administration details
    """
    try:
        # Establish database connection
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Enable foreign key constraint enforcement (important for data integrity)
        cursor.execute('PRAGMA foreign_keys = ON')
        
        # ----------------------------------------------------------------
        # TABLE 1: Doctors
        # Stores information about medical staff including their specialty
        # and shift schedules for workload analysis
        # ----------------------------------------------------------------
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Doctors (
                doctor_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                specialty TEXT NOT NULL,
                shift_start TEXT NOT NULL,
                shift_end TEXT NOT NULL,
                years_experience INTEGER
            )
        ''')
        
        # ----------------------------------------------------------------
        # TABLE 2: Patients
        # Contains patient demographic information and links to assigned doctor
        # Foreign key ensures referential integrity with Doctors table
        # ----------------------------------------------------------------
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Patients (
                patient_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                age INTEGER CHECK(age >= 0 AND age <= 120),
                gender TEXT CHECK(gender IN ('Male', 'Female', 'Other')),
                admission_date TEXT NOT NULL,
                diagnosis TEXT,
                doctor_id INTEGER NOT NULL,
                FOREIGN KEY (doctor_id) REFERENCES Doctors(doctor_id)
            )
        ''')
        
        # ----------------------------------------------------------------
        # TABLE 3: VitalsLog
        # Time-series data tracking patient vital signs over time
        # Critical for identifying trends and high-risk patients
        # ----------------------------------------------------------------
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS VitalsLog (
                vitals_id INTEGER PRIMARY KEY AUTOINCREMENT,
                patient_id INTEGER NOT NULL,
                timestamp TEXT NOT NULL,
                heart_rate INTEGER CHECK(heart_rate >= 0 AND heart_rate <= 300),
                blood_pressure TEXT,
                temperature REAL CHECK(temperature >= 30.0 AND temperature <= 45.0),
                oxygen_saturation INTEGER CHECK(oxygen_saturation >= 0 AND oxygen_saturation <= 100),
                FOREIGN KEY (patient_id) REFERENCES Patients(patient_id)
            )
        ''')
        
        # ----------------------------------------------------------------
        # TABLE 4: Medications
        # Tracks prescribed medications and administration records
        # Enables analysis of medication usage patterns
        # ----------------------------------------------------------------
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS Medications (
                med_id INTEGER PRIMARY KEY AUTOINCREMENT,
                patient_id INTEGER NOT NULL,
                medication_name TEXT NOT NULL,
                dosage TEXT NOT NULL,
                frequency TEXT NOT NULL,
                administered_date TEXT NOT NULL,
                prescribed_by INTEGER,
                FOREIGN KEY (patient_id) REFERENCES Patients(patient_id),
                FOREIGN KEY (prescribed_by) REFERENCES Doctors(doctor_id)
            )
        ''')
        
        # Create indexes for better query performance on frequently accessed columns
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_vitals_patient ON VitalsLog(patient_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_vitals_timestamp ON VitalsLog(timestamp)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_meds_patient ON Medications(patient_id)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_patients_doctor ON Patients(doctor_id)')
        
        # Commit all changes to the database
        conn.commit()
        print("✓ Database schema created successfully!")
        print("✓ Tables created: Doctors, Patients, VitalsLog, Medications")
        print("✓ Indexes created for optimized queries")
        
    except sqlite3.Error as e:
        # Handle any database errors
        print(f"✗ Database error occurred: {e}")
        raise
    finally:
        # Always close the connection to free resources
        if conn:
            conn.close()

# Execute the schema creation function
create_database_schema()

In [ ]:
# ============================================================================
# CELL 4: BATCH DATA SIMULATION AND GENERATION
# ============================================================================
# This cell generates large volumes of realistic synthetic healthcare data
# to populate the database. It demonstrates batch processing techniques and
# creates representative data for testing queries and visualizations.
# ============================================================================

def generate_mock_data(num_patients=1000, num_doctors=50, num_vitals_per_patient=15, num_meds_per_patient=5):
    """
    Generates comprehensive synthetic healthcare data using batch processing.
    
    This function creates realistic mock data for all database tables, ensuring
    referential integrity and realistic patterns (e.g., vitals that deteriorate
    or improve over time, appropriate medication frequencies).
    
    Args:
        num_patients (int): Number of patient records to generate (default: 1000)
        num_doctors (int): Number of doctor records to generate (default: 50)
        num_vitals_per_patient (int): Vital sign entries per patient (default: 15)
        num_meds_per_patient (int): Medication records per patient (default: 5)
    
    Returns:
        dict: Dictionary containing DataFrames for each table:
              {'doctors': df, 'patients': df, 'vitals': df, 'medications': df}
    """
    
    print(f"Generating mock data for {num_patients} patients...")
    
    # ----------------------------------------------------------------
    # GENERATE DOCTORS DATA
    # Create realistic doctor profiles with various specialties and shifts
    # ----------------------------------------------------------------
    doctor_first_names = ['James', 'Mary', 'John', 'Patricia', 'Robert', 'Jennifer', 
                         'Michael', 'Linda', 'William', 'Elizabeth', 'David', 'Barbara',
                         'Richard', 'Susan', 'Joseph', 'Jessica', 'Thomas', 'Sarah']
    doctor_last_names = ['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Garcia',
                        'Miller', 'Davis', 'Rodriguez', 'Martinez', 'Hernandez', 'Lopez',
                        'Wilson', 'Anderson', 'Taylor', 'Thomas', 'Moore', 'Jackson']
    
    specialties = ['Cardiology', 'Neurology', 'Oncology', 'Pediatrics', 'General Medicine',
                  'Emergency Medicine', 'Internal Medicine', 'Surgery', 'Psychiatry']
    
    # Shift schedules (start hour, end hour)
    shifts = [('06:00', '14:00'), ('14:00', '22:00'), ('22:00', '06:00'),  # 8-hour shifts
              ('08:00', '20:00'), ('20:00', '08:00')]  # 12-hour shifts
    
    doctors_data = []
    for i in range(num_doctors):
        shift = random.choice(shifts)
        doctors_data.append({
            'name': f'Dr. {random.choice(doctor_first_names)} {random.choice(doctor_last_names)}',
            'specialty': random.choice(specialties),
            'shift_start': shift[0],
            'shift_end': shift[1],
            'years_experience': random.randint(1, 35)
        })
    
    doctors_df = pd.DataFrame(doctors_data)
    
    # ----------------------------------------------------------------
    # GENERATE PATIENTS DATA
    # Create diverse patient demographics with realistic distributions
    # ----------------------------------------------------------------
    diagnoses = ['Hypertension', 'Diabetes Type 2', 'Coronary Artery Disease', 'Heart Failure',
                'Pneumonia', 'COPD', 'Cancer', 'Stroke', 'Sepsis', 'Kidney Disease']
    
    patients_data = []
    for i in range(num_patients):
        # Age distribution weighted towards older patients (hospital demographic)
        age = int(np.random.normal(65, 15))
        age = max(18, min(95, age))  # Clamp between 18-95
        
        patients_data.append({
            'name': f'Patient_{i+1:04d}',
            'age': age,
            'gender': random.choice(['Male', 'Female']),
            'admission_date': (datetime.now() - timedelta(days=random.randint(1, 90))).strftime('%Y-%m-%d'),
            'diagnosis': random.choice(diagnoses),
            'doctor_id': random.randint(1, num_doctors)
        })
    
    patients_df = pd.DataFrame(patients_data)
    
    # ----------------------------------------------------------------
    # GENERATE VITALS LOG DATA
    # Create time-series vital signs with realistic trends and variations
    # ----------------------------------------------------------------
    vitals_data = []
    
    for patient_id in range(1, num_patients + 1):
        # Determine if patient is improving, stable, or deteriorating
        trend = random.choice(['improving', 'stable', 'deteriorating'])
        
        # Initial vital signs (some patients start with abnormal values)
        if trend == 'deteriorating':
            base_hr = random.randint(70, 90)  # Start normal
        elif trend == 'improving':
            base_hr = random.randint(95, 115)  # Start high
        else:
            base_hr = random.randint(70, 85)  # Stable
        
        # Generate vitals over time with trends
        for check in range(num_vitals_per_patient):
            # Time progresses backwards (most recent first when sorted DESC)
            timestamp = datetime.now() - timedelta(hours=check * 8)
            
            # Apply trend to heart rate
            if trend == 'improving':
                hr_modifier = -check * 2  # Heart rate decreases over time
            elif trend == 'deteriorating':
                hr_modifier = check * 3  # Heart rate increases over time
            else:
                hr_modifier = random.randint(-3, 3)  # Random variation
            
            heart_rate = max(50, min(150, base_hr + hr_modifier + random.randint(-5, 5)))
            
            # Blood pressure (systolic/diastolic)
            systolic = random.randint(100, 160)
            diastolic = random.randint(60, 100)
            
            # Temperature in Celsius (normal range with some fever cases)
            temp = round(random.uniform(36.2, 38.5), 1)
            
            # Oxygen saturation (SpO2)
            o2_sat = random.randint(92, 100)
            
            vitals_data.append({
                'patient_id': patient_id,
                'timestamp': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                'heart_rate': heart_rate,
                'blood_pressure': f'{systolic}/{diastolic}',
                'temperature': temp,
                'oxygen_saturation': o2_sat
            })
    
    vitals_df = pd.DataFrame(vitals_data)
    
    # ----------------------------------------------------------------
    # GENERATE MEDICATIONS DATA
    # Create medication records with appropriate dosages and frequencies
    # ----------------------------------------------------------------
    medications_list = [
        ('Aspirin', ['81mg', '325mg'], ['Once daily', 'Twice daily']),
        ('Metformin', ['500mg', '850mg', '1000mg'], ['Twice daily', 'Three times daily']),
        ('Lisinopril', ['10mg', '20mg', '40mg'], ['Once daily']),
        ('Atorvastatin', ['10mg', '20mg', '40mg', '80mg'], ['Once daily']),
        ('Levothyroxine', ['25mcg', '50mcg', '100mcg'], ['Once daily']),
        ('Amlodipine', ['5mg', '10mg'], ['Once daily']),
        ('Omeprazole', ['20mg', '40mg'], ['Once daily']),
        ('Metoprolol', ['25mg', '50mg', '100mg'], ['Twice daily']),
        ('Albuterol', ['90mcg'], ['As needed', 'Four times daily']),
        ('Furosemide', ['20mg', '40mg', '80mg'], ['Once daily', 'Twice daily'])
    ]
    
    meds_data = []
    for patient_id in range(1, num_patients + 1):
        # Each patient gets a random selection of medications
        patient_meds = random.sample(medications_list, min(num_meds_per_patient, len(medications_list)))
        
        for med_name, dosages, frequencies in patient_meds:
            meds_data.append({
                'patient_id': patient_id,
                'medication_name': med_name,
                'dosage': random.choice(dosages),
                'frequency': random.choice(frequencies),
                'administered_date': (datetime.now() - timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d'),
                'prescribed_by': random.randint(1, num_doctors)
            })
    
    meds_df = pd.DataFrame(meds_data)
    
    print(f"✓ Generated {len(doctors_df)} doctors")
    print(f"✓ Generated {len(patients_df)} patients")
    print(f"✓ Generated {len(vitals_df)} vital sign records")
    print(f"✓ Generated {len(meds_df)} medication records")
    
    return {
        'doctors': doctors_df,
        'patients': patients_df,
        'vitals': vitals_df,
        'medications': meds_df
    }

def insert_data_to_db(data_dict, db_path='healthcare.db'):
    """
    Inserts generated data into the database using batch operations.
    
    Uses pandas to_sql() method for efficient batch insertion of large datasets.
    This is much faster than individual INSERT statements.
    
    Args:
        data_dict (dict): Dictionary containing DataFrames for each table
        db_path (str): Path to the SQLite database file
    
    Returns:
        None
    """
    try:
        conn = sqlite3.connect(db_path)
        
        # Insert data in correct order (respecting foreign key constraints)
        # Doctors must be inserted before Patients (Patients reference Doctors)
        print("Inserting doctors...")
        data_dict['doctors'].to_sql('Doctors', conn, if_exists='append', index=False)
        
        print("Inserting patients...")
        data_dict['patients'].to_sql('Patients', conn, if_exists='append', index=False)
        
        # VitalsLog and Medications reference Patients, so insert them last
        print("Inserting vitals logs...")
        data_dict['vitals'].to_sql('VitalsLog', conn, if_exists='append', index=False)
        
        print("Inserting medications...")
        data_dict['medications'].to_sql('Medications', conn, if_exists='append', index=False)
        
        conn.close()
        print("\n✓ All data inserted successfully using batch processing!")
        
    except sqlite3.Error as e:
        print(f"✗ Error inserting data: {e}")
        raise

# ============================================================================
# EXECUTE DATA GENERATION AND INSERTION
# ============================================================================
# Generate large-scale mock data (1000 patients with 15 vitals checks each = 15,000 records)
mock_data = generate_mock_data(
    num_patients=1000,
    num_doctors=50,
    num_vitals_per_patient=15,
    num_meds_per_patient=5
)

# Insert all data into database
insert_data_to_db(mock_data)

print("\nDatabase is now populated and ready for queries!")

In [ ]:
# ============================================================================
# CELL 5: ADVANCED SQL QUERIES WITH AGGREGATIONS, JOINS, AND WINDOW FUNCTIONS
# ============================================================================
# This cell implements complex SQL queries demonstrating:
# - Multi-table JOINs
# - Aggregation functions (AVG, COUNT, SUM)
# - Window functions for advanced analytics
# - Subqueries for complex filtering
# ============================================================================

def get_high_risk_patients(db_path='healthcare.db', threshold=100, num_recent_checks=3):
    """
    Identifies high-risk patients based on average heart rate in recent vitals checks.
    
    This query uses:
    - Window function (ROW_NUMBER) to get the most recent N checks per patient
    - Subquery to filter only recent vitals
    - JOIN to combine patient and vitals data
    - Aggregation (AVG) to calculate average heart rate
    - HAVING clause to filter results
    
    Args:
        db_path (str): Path to the database
        threshold (int): Heart rate threshold for high-risk classification
        num_recent_checks (int): Number of recent vitals to consider
    
    Returns:
        pd.DataFrame: High-risk patients with their average heart rate
    """
    conn = sqlite3.connect(db_path)
    
    # Complex SQL query using window function and subquery
    query = '''
    WITH RecentVitals AS (
        -- Use ROW_NUMBER window function to rank vitals by timestamp for each patient
        SELECT 
            patient_id,
            heart_rate,
            timestamp,
            ROW_NUMBER() OVER (PARTITION BY patient_id ORDER BY timestamp DESC) as rn
        FROM VitalsLog
    )
    SELECT 
        p.patient_id,
        p.name,
        p.age,
        p.diagnosis,
        AVG(rv.heart_rate) as avg_heart_rate,
        COUNT(rv.heart_rate) as num_checks,
        MAX(rv.heart_rate) as max_heart_rate,
        MIN(rv.heart_rate) as min_heart_rate
    FROM Patients p
    JOIN RecentVitals rv ON p.patient_id = rv.patient_id
    WHERE rv.rn <= ?  -- Only consider the N most recent checks
    GROUP BY p.patient_id, p.name, p.age, p.diagnosis
    HAVING AVG(rv.heart_rate) > ?  -- Filter for high average heart rate
    ORDER BY avg_heart_rate DESC
    '''
    
    df = pd.read_sql_query(query, conn, params=(num_recent_checks, threshold))
    conn.close()
    
    print(f"Found {len(df)} high-risk patients with avg heart rate > {threshold} bpm")
    return df

def get_busiest_shifts(db_path='healthcare.db'):
    """
    Analyzes doctor workload by calculating vitals checks per hour.
    
    This query demonstrates:
    - Multi-table JOIN (VitalsLog -> Patients -> Doctors)
    - Date/time functions (strftime) to extract hour
    - Aggregation (COUNT) to sum activities
    - GROUP BY with complex expressions
    
    Args:
        db_path (str): Path to the database
    
    Returns:
        pd.DataFrame: Hours of day with vitals check counts
    """
    conn = sqlite3.connect(db_path)
    
    query = '''
    SELECT 
        strftime('%H', v.timestamp) as hour_of_day,
        COUNT(*) as total_checks,
        COUNT(DISTINCT p.patient_id) as unique_patients,
        COUNT(DISTINCT d.doctor_id) as active_doctors,
        ROUND(AVG(v.heart_rate), 1) as avg_heart_rate
    FROM VitalsLog v
    JOIN Patients p ON v.patient_id = p.patient_id
    JOIN Doctors d ON p.doctor_id = d.doctor_id
    GROUP BY hour_of_day
    ORDER BY total_checks DESC
    '''
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    print(f"Analyzed {len(df)} hours of activity")
    return df

def get_medication_usage_stats(db_path='healthcare.db'):
    """
    Calculates comprehensive medication usage statistics.
    
    Demonstrates:
    - Multiple aggregations (COUNT, SUM via COUNT DISTINCT)
    - GROUP BY for categorical analysis
    - ORDER BY with aggregation results
    
    Args:
        db_path (str): Path to the database
    
    Returns:
        pd.DataFrame: Medication statistics
    """
    conn = sqlite3.connect(db_path)
    
    query = '''
    SELECT 
        medication_name,
        COUNT(*) as total_prescriptions,
        COUNT(DISTINCT patient_id) as unique_patients,
        COUNT(DISTINCT prescribed_by) as prescribing_doctors,
        -- Most common dosage for this medication
        (SELECT dosage 
         FROM Medications m2 
         WHERE m2.medication_name = m.medication_name 
         GROUP BY dosage 
         ORDER BY COUNT(*) DESC 
         LIMIT 1) as most_common_dosage
    FROM Medications m
    GROUP BY medication_name
    ORDER BY total_prescriptions DESC
    '''
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    print(f"Analyzed {len(df)} different medications")
    return df

def get_patient_recovery_trends(db_path='healthcare.db', patient_id=1):
    """
    Analyzes recovery trends for a specific patient over time.
    
    Uses:
    - Window function (LAG) to calculate change from previous reading
    - Multi-column selection with calculations
    - Time-series ordering
    
    Args:
        db_path (str): Path to the database
        patient_id (int): Patient to analyze
    
    Returns:
        pd.DataFrame: Vitals over time with trend indicators
    """
    conn = sqlite3.connect(db_path)
    
    query = '''
    SELECT 
        v.timestamp,
        v.heart_rate,
        v.temperature,
        v.oxygen_saturation,
        -- Calculate change from previous reading using LAG window function
        v.heart_rate - LAG(v.heart_rate) OVER (ORDER BY v.timestamp) as hr_change,
        -- Moving average of last 3 readings
        AVG(v.heart_rate) OVER (
            ORDER BY v.timestamp 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) as hr_moving_avg
    FROM VitalsLog v
    WHERE v.patient_id = ?
    ORDER BY v.timestamp ASC
    '''
    
    df = pd.read_sql_query(query, conn, params=(patient_id,))
    conn.close()
    
    return df

def get_doctor_workload_analysis(db_path='healthcare.db'):
    """
    Comprehensive doctor workload analysis.
    
    Demonstrates:
    - Complex multi-table JOINs
    - Multiple aggregations per group
    - Subquery for additional calculations
    
    Args:
        db_path (str): Path to the database
    
    Returns:
        pd.DataFrame: Doctor workload metrics
    """
    conn = sqlite3.connect(db_path)
    
    query = '''
    SELECT 
        d.name as doctor_name,
        d.specialty,
        d.shift_start || ' - ' || d.shift_end as shift,
        COUNT(DISTINCT p.patient_id) as total_patients,
        COUNT(v.vitals_id) as total_vitals_checks,
        COUNT(m.med_id) as total_prescriptions,
        ROUND(AVG(p.age), 1) as avg_patient_age,
        -- Count of high-risk patients (avg HR > 100 in last 3 checks)
        (SELECT COUNT(DISTINCT p2.patient_id)
         FROM Patients p2
         JOIN VitalsLog v2 ON p2.patient_id = v2.patient_id
         WHERE p2.doctor_id = d.doctor_id
         AND v2.vitals_id IN (
             SELECT vitals_id FROM VitalsLog v3
             WHERE v3.patient_id = p2.patient_id
             ORDER BY v3.timestamp DESC
             LIMIT 3
         )
         GROUP BY p2.patient_id
         HAVING AVG(v2.heart_rate) > 100
        ) as high_risk_patients
    FROM Doctors d
    LEFT JOIN Patients p ON d.doctor_id = p.doctor_id
    LEFT JOIN VitalsLog v ON p.patient_id = v.patient_id
    LEFT JOIN Medications m ON p.patient_id = m.patient_id
    GROUP BY d.doctor_id, d.name, d.specialty, d.shift_start, d.shift_end
    ORDER BY total_vitals_checks DESC
    '''
    
    df = pd.read_sql_query(query, conn)
    conn.close()
    
    print(f"Analyzed workload for {len(df)} doctors")
    return df

# ============================================================================
# EXECUTE SQL QUERIES AND DISPLAY RESULTS
# ============================================================================

print("=" * 70)
print("EXECUTING ADVANCED SQL QUERIES")
print("=" * 70)

# Query 1: High-risk patients
print("\n1. HIGH-RISK PATIENTS (Using Window Functions and Aggregations)")
print("-" * 70)
high_risk = get_high_risk_patients(threshold=100, num_recent_checks=3)
display(high_risk.head(10))

# Query 2: Busiest shift hours
print("\n2. BUSIEST SHIFT HOURS (Multi-table JOINs and Aggregations)")
print("-" * 70)
busiest = get_busiest_shifts()
display(busiest.head(10))

# Query 3: Medication usage statistics
print("\n3. MEDICATION USAGE STATISTICS (Subqueries and Aggregations)")
print("-" * 70)
med_stats = get_medication_usage_stats()
display(med_stats.head(10))

# Query 4: Doctor workload analysis
print("\n4. DOCTOR WORKLOAD ANALYSIS (Complex JOINs and Nested Subqueries)")
print("-" * 70)
workload = get_doctor_workload_analysis()
display(workload.head(10))

print("\n" + "=" * 70)
print("✓ All SQL queries executed successfully!")
print("=" * 70)

In [ ]:
# ============================================================================
# CELL 6: INTERACTIVE DASHBOARD WITH MULTIPLE VISUALIZATIONS
# ============================================================================
# This cell creates a comprehensive interactive dashboard featuring:
# - Patient recovery trend visualization (line charts)
# - Medication usage frequency (bar charts)
# - High-risk patient alerts
# - Busiest shift hours visualization
# - Interactive filtering with widgets
# ============================================================================

def create_comprehensive_dashboard(db_path='healthcare.db'):
    """
    Creates an interactive healthcare monitoring dashboard.
    
    This dashboard provides multiple interactive visualizations allowing
    healthcare professionals to:
    - Monitor individual patient recovery trends
    - View medication usage patterns
    - Identify high-risk patients
    - Analyze shift workload distribution
    
    Args:
        db_path (str): Path to the SQLite database
    
    Returns:
        None (displays interactive widgets)
    """
    
    # ----------------------------------------------------------------
    # LOAD DATA FROM DATABASE
    # ----------------------------------------------------------------
    conn = sqlite3.connect(db_path)
    
    # Load all required data
    patients = pd.read_sql_query("SELECT * FROM Patients ORDER BY name", conn)
    vitals = pd.read_sql_query("SELECT * FROM VitalsLog ORDER BY timestamp", conn)
    medications = pd.read_sql_query("SELECT * FROM Medications", conn)
    doctors = pd.read_sql_query("SELECT * FROM Doctors", conn)
    
    conn.close()
    
    # Convert timestamp strings to datetime for better plotting
    vitals['timestamp'] = pd.to_datetime(vitals['timestamp'])
    
    # ----------------------------------------------------------------
    # CREATE INTERACTIVE WIDGETS
    # ----------------------------------------------------------------
    
    # Patient selector dropdown
    patient_dropdown = widgets.Dropdown(
        options=[(row['name'], row['patient_id']) for _, row in patients.iterrows()],
        description='Patient:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # Heart rate threshold slider for high-risk detection
    threshold_slider = widgets.IntSlider(
        value=100,
        min=80,
        max=120,
        step=5,
        description='HR Threshold:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    
    # Date range selector for filtering vitals
    date_range_slider = widgets.IntRangeSlider(
        value=[0, 30],
        min=0,
        max=90,
        step=1,
        description='Days Back:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px')
    )
    
    # Specialty filter for doctor workload
    specialty_dropdown = widgets.Dropdown(
        options=['All'] + sorted(doctors['specialty'].unique().tolist()),
        description='Specialty:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )
    
    # Output widget for displaying charts
    output = widgets.Output()
    
    # ----------------------------------------------------------------
    # DASHBOARD UPDATE FUNCTION
    # ----------------------------------------------------------------
    
    def update_dashboard(change):
        """
        Updates all dashboard visualizations based on widget selections.
        
        This function is called whenever any widget value changes,
        regenerating all charts with the new filter parameters.
        """
        with output:
            clear_output(wait=True)
            
            # Get selected values from widgets
            selected_patient_id = patient_dropdown.value
            hr_threshold = threshold_slider.value
            days_start, days_end = date_range_slider.value
            selected_specialty = specialty_dropdown.value
            
            # Calculate date range for filtering
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days_end)
            filter_end = end_date - timedelta(days=days_start)
            
            # Display dashboard title
            print("\n" + "=" * 80)
            print("HEALTHCARE PATIENT MONITORING DASHBOARD".center(80))
            print("=" * 80 + "\n")
            
            # ----------------------------------------------------------------
            # VISUALIZATION 1: PATIENT RECOVERY TRENDS
            # Line chart showing vital signs over time for selected patient
            # ----------------------------------------------------------------
            patient_vitals = vitals[
                (vitals['patient_id'] == selected_patient_id) &
                (vitals['timestamp'] >= start_date) &
                (vitals['timestamp'] <= filter_end)
            ].sort_values('timestamp')
            
            if len(patient_vitals) > 0:
                # Get patient information
                patient_info = patients[patients['patient_id'] == selected_patient_id].iloc[0]
                
                print(f"📊 Patient: {patient_info['name']}")
                print(f"   Age: {patient_info['age']} | Gender: {patient_info['gender']}")
                print(f"   Diagnosis: {patient_info['diagnosis']}")
                print(f"   Admission: {patient_info['admission_date']}\n")
                
                # Create subplot with multiple vital signs
                fig = make_subplots(
                    rows=3, cols=1,
                    subplot_titles=('Heart Rate Over Time', 'Temperature Trend', 'Oxygen Saturation'),
                    vertical_spacing=0.1
                )
                
                # Heart rate trend with threshold line
                fig.add_trace(
                    go.Scatter(
                        x=patient_vitals['timestamp'],
                        y=patient_vitals['heart_rate'],
                        mode='lines+markers',
                        name='Heart Rate',
                        line=dict(color='red', width=2),
                        marker=dict(size=6)
                    ),
                    row=1, col=1
                )
                
                # Add threshold line
                fig.add_hline(
                    y=hr_threshold, 
                    line_dash="dash", 
                    line_color="orange",
                    annotation_text=f"Risk Threshold: {hr_threshold}",
                    row=1, col=1
                )
                
                # Temperature trend
                fig.add_trace(
                    go.Scatter(
                        x=patient_vitals['timestamp'],
                        y=patient_vitals['temperature'],
                        mode='lines+markers',
                        name='Temperature (°C)',
                        line=dict(color='blue', width=2),
                        marker=dict(size=6)
                    ),
                    row=2, col=1
                )
                
                # Oxygen saturation trend
                fig.add_trace(
                    go.Scatter(
                        x=patient_vitals['timestamp'],
                        y=patient_vitals['oxygen_saturation'],
                        mode='lines+markers',
                        name='SpO2 (%)',
                        line=dict(color='green', width=2),
                        marker=dict(size=6)
                    ),
                    row=3, col=1
                )
                
                # Update layout
                fig.update_layout(
                    height=900,
                    title_text=f"Recovery Trends for {patient_info['name']}",
                    showlegend=False
                )
                
                fig.update_xaxes(title_text="Time", row=3, col=1)
                fig.update_yaxes(title_text="BPM", row=1, col=1)
                fig.update_yaxes(title_text="°C", row=2, col=1)
                fig.update_yaxes(title_text="%", row=3, col=1)
                
                fig.show()
                
                # Show statistics
                print("\n📈 Vital Sign Statistics:")
                print(f"   Heart Rate:  Avg={patient_vitals['heart_rate'].mean():.1f} bpm, "
                      f"Min={patient_vitals['heart_rate'].min()}, Max={patient_vitals['heart_rate'].max()}")
                print(f"   Temperature: Avg={patient_vitals['temperature'].mean():.1f}°C, "
                      f"Min={patient_vitals['temperature'].min()}, Max={patient_vitals['temperature'].max()}")
                print(f"   SpO2:        Avg={patient_vitals['oxygen_saturation'].mean():.1f}%, "
                      f"Min={patient_vitals['oxygen_saturation'].min()}, Max={patient_vitals['oxygen_saturation'].max()}")
            else:
                print("⚠️ No vital signs data available for selected date range.")
            
            print("\n" + "-" * 80 + "\n")
            
            # ----------------------------------------------------------------
            # VISUALIZATION 2: MEDICATION USAGE FREQUENCY
            # Bar chart showing most prescribed medications
            # ----------------------------------------------------------------
            med_counts = medications['medication_name'].value_counts().reset_index()
            med_counts.columns = ['Medication', 'Count']
            
            fig2 = px.bar(
                med_counts.head(10),
                x='Medication',
                y='Count',
                title='Top 10 Most Prescribed Medications',
                labels={'Count': 'Number of Prescriptions'},
                color='Count',
                color_continuous_scale='Blues'
            )
            
            fig2.update_layout(height=500, showlegend=False)
            fig2.show()
            
            print("\n" + "-" * 80 + "\n")
            
            # ----------------------------------------------------------------
            # VISUALIZATION 3: HIGH-RISK PATIENT ALERTS
            # Display patients exceeding heart rate threshold
            # ----------------------------------------------------------------
            high_risk = get_high_risk_patients(db_path, threshold=hr_threshold, num_recent_checks=3)
            
            if len(high_risk) > 0:
                print(f"⚠️  HIGH-RISK PATIENT ALERTS (HR > {hr_threshold} bpm)")
                print(f"    Found {len(high_risk)} patients requiring attention:\n")
                
                # Create alert visualization
                fig3 = px.bar(
                    high_risk.head(15),
                    x='name',
                    y='avg_heart_rate',
                    title=f'High-Risk Patients (Average HR > {hr_threshold} bpm)',
                    labels={'name': 'Patient', 'avg_heart_rate': 'Average Heart Rate (bpm)'},
                    color='avg_heart_rate',
                    color_continuous_scale='Reds'
                )
                
                fig3.add_hline(
                    y=hr_threshold,
                    line_dash="dash",
                    line_color="black",
                    annotation_text="Threshold"
                )
                
                fig3.update_layout(height=500)
                fig3.show()
            else:
                print(f"✅ No high-risk patients detected (all patients below {hr_threshold} bpm threshold)")
            
            print("\n" + "-" * 80 + "\n")
            
            # ----------------------------------------------------------------
            # VISUALIZATION 4: BUSIEST SHIFT HOURS
            # Bar chart showing workload distribution across hours
            # ----------------------------------------------------------------
            busiest = get_busiest_shifts(db_path)
            busiest['hour_of_day'] = busiest['hour_of_day'].astype(int)
            
            fig4 = px.bar(
                busiest,
                x='hour_of_day',
                y='total_checks',
                title='Busiest Shift Hours - Vitals Monitoring Activity',
                labels={'hour_of_day': 'Hour of Day (24h)', 'total_checks': 'Number of Vitals Checks'},
                color='total_checks',
                color_continuous_scale='Viridis'
            )
            
            fig4.update_layout(height=500, showlegend=False)
            fig4.update_xaxes(tickmode='linear', tick0=0, dtick=1)
            fig4.show()
            
            # Show peak hours
            peak_hour = busiest.iloc[0]
            print(f"\n🕐 Peak Activity Hour: {int(peak_hour['hour_of_day']):02d}:00")
            print(f"   Total Checks: {peak_hour['total_checks']}")
            print(f"   Unique Patients: {peak_hour['unique_patients']}")
            print(f"   Active Doctors: {peak_hour['active_doctors']}")
            
            print("\n" + "=" * 80)
            print("Dashboard refresh complete!".center(80))
            print("=" * 80)
    
    # ----------------------------------------------------------------
    # ATTACH WIDGET OBSERVERS
    # ----------------------------------------------------------------
    patient_dropdown.observe(update_dashboard, names='value')
    threshold_slider.observe(update_dashboard, names='value')
    date_range_slider.observe(update_dashboard, names='value')
    specialty_dropdown.observe(update_dashboard, names='value')
    
    # ----------------------------------------------------------------
    # DISPLAY DASHBOARD
    # ----------------------------------------------------------------
    print("\n" + "=" * 80)
    print("INTERACTIVE HEALTHCARE DASHBOARD".center(80))
    print("=" * 80)
    print("\nUse the controls below to filter and explore the data:\n")
    
    # Create control panel layout
    controls = widgets.VBox([
        widgets.HTML("<h3>Dashboard Controls</h3>"),
        widgets.HBox([patient_dropdown, specialty_dropdown]),
        threshold_slider,
        date_range_slider,
        widgets.HTML("<hr>")
    ])
    
    # Display controls and output
    display(controls)
    display(output)
    
    # Initial dashboard render
    update_dashboard(None)

# ============================================================================
# LAUNCH INTERACTIVE DASHBOARD
# ============================================================================
create_comprehensive_dashboard()

---

## Reflection

### Challenges Faced

**SQL Complexity**: The most significant challenge was implementing the high-risk patient detection query using window functions. SQLite's `ROW_NUMBER() OVER (PARTITION BY patient_id ORDER BY timestamp DESC)` required careful structuring with CTEs to correctly identify the last 3 vitals checks per patient. Initially, subqueries with `LIMIT` didn't work as expected across multiple patients, requiring a shift to window functions for accurate results.

**Data Visualization**: Creating the interactive dashboard with synchronized filtering across multiple visualizations was complex. Coordinating ipywidgets observers to update all four charts simultaneously while maintaining performance required careful state management. The three-panel subplot for recovery trends (heart rate, temperature, SpO2) needed precise layout configuration with `make_subplots` to ensure readability.

**Data Realism**: Generating realistic time-series vitals with meaningful trends (improving/stable/deteriorating patterns) required thoughtful modeling. Ensuring proper foreign key relationships during batch insertion while maintaining referential integrity added complexity to the data generation pipeline.

Despite these challenges, the final implementation successfully demonstrates advanced database concepts with a professional, interactive healthcare monitoring system.

---
